# Tutorial 02:  Inside the "optimize"

In [5]:
#!/usr/bin/env python3

# Standard python library

# Local library:
import IO.user_provided
import objective.setup_objective
import optimizer.gradient_free
import objective.compute_objective

# Third party library:

----------------------------------------------------------------------------------
###                       Part 1: taking the input from the command-line interface                         
----------------------------------------------------------------------------------
#### Purposes: 
* obtain the input parameters from the command-line interface

#### Return:
* main_logger: an object that defines the log file output (you don't have to do anything with it; all the logging information from other modules will be sent to the main logger object)
* TOTAL_CORES: Number of cores assigned by slurm scheduler (int)
* INPUT: The input file name (str)
* JOBID: a user-provided name typically a combination of Slurm job id and user-provided name (str)
* Ref: Reference data folder path (str)
* prep: template folder path for running sampling (str) 

In [6]:
# For the interactive job: (For Tutorial only):
(main_logger,
 TOTAL_CORES,
 INPUT,
 JOBID,
 Ref,
 prep) = IO.user_provided.from_command_line(jobID="tutorial",
                                           total_cores=2,
                                           input_file="in_obj",
                                           ref_address="../force_matching_tutorial/ReferenceData",
                                           prep_address="../force_matching_tutorial/prepsystem").finish_reading()


# Running the program from command (uncomment for the normal use): 
"""
main_logger, TOTAL_CORES, INPUT, JOBID, Ref, prep = (IO
                                           .user_provided
                                           .from_command_line()
                                           .finish_reading())
""" 
print ("cores requested:", TOTAL_CORES,type(TOTAL_CORES))
print ("input file name:", INPUT,type(INPUT))
print ("Job id:", JOBID,type(JOBID))
print ("Reference data path:", Ref,type(Ref))
print ("prepsystem data path:", prep,type(prep))

cores requested: 2 <class 'int'>
input file name: in_obj <class 'str'>
Job id: tutorial <class 'str'>
Reference data path: ../force_matching_tutorial/ReferenceData <class 'str'>
prepsystem data path: ../force_matching_tutorial/prepsystem <class 'str'>


----------------------------------------------------------------------------------
###                           Part 2: set up the workflow                                
----------------------------------------------------------------------------------
#### Purposes: 
* Set up the job folder including "Predicted", "Output" and "Restart" folders inside it 
* Initialize a sampling method such LAMMPS
* Parse the arugment of objective functions from the input file

#### Return: 
* ref_dict: a python dictionary containing reference data address. Each key is the name of matching type(force,rdf ...). Each value is another dictionary containing the arugment for the sub folder in that matching type. 
* predict_dict: a python dictionary containg predicted data address. Same data structure as ref_dict
* argument_dict: a python dictionary containing arguments needed to run objective functions inclduing number of cores requested, buffersize etc ... 
* LAMMPS: a Python class instance with the method: "run", to invoke simulators to perform the sampling as subprocesses and a method: "exit" to check the status of all jobs
* last_line: the stopping number of lines in the input file.This includes all the objective function definitions, and commands to be invoked. Optimizer will continue from here to read the rest of input. 

In [10]:
(ref_dict,
 predict_dict,
 argument_dict,
 LAMMPS,
 last_line) = objective.setup_objective.setup(INPUT,
                                              TOTAL_CORES,
                                              JOBID,
                                              overwrite=True,
                                              Ref_folder=Ref,
                                              prep_folder=prep).finish()

# display the data structure:
print ("referenced data dictionary:\n",ref_dict)

print ("predict data dictionary:\n",predict_dict)

print ("argument dictionary:",argument_dict)

referenced data dictionary:
 {'force': {'mW_300K_1bar_500': ('/project/palmer/Jingxiang/ours_optimization/tutorial/force_matching_tutorial/ReferenceData/force/mW_300K_1bar_500',)}}
predict data dictionary:
 {'force': {'mW_300K_1bar_500': ('/project/palmer/Jingxiang/ours_optimization/tutorial/Tutorial_01_GettingStarted/tutorial/Predicted/force/mW_300K_1bar_500',)}}
argument dictionary: {'force': {'mW_300K_1bar_500': ('mW_300K_1bar_500', 1.0, 2, 2, 'bf 5000 eng abs w 0.0 1.0')}}


----------------------------------------------------------------------------------
###                          Part 3: initialize objective functions                     
----------------------------------------------------------------------------------
#### Purposes: 
* The "eval_objective" instance contains a list of objects. Each object corresponds to an instance of each type of objective function's "load" class. The "load" class provides the detailed steps of how to compute objective function and thus, it is defined uniquely for each type of matching e.g. force matching, rdf matching etc. The "eval_objective" will be passed to an optimizer object, and it will be invoked in every iteration to compute the overall objective function values (sums of weighted objective function values return by each load class)

#### Return: 
* The "eval_objective" contains a method: "optimize" that takes an optimized set of force-field parameters, perform the sampling to compute the predicted properties, and return objective function values by comparing with reference properties. The method "optimize" takes "potential type", "force-field parameters", and "status" as arguments. Another method: "update" is optional but is used to update the currently best predicted properties, which will be dumped to "Output" folder in the job folder.

In [ ]:
eval_objective = (objective
                  .compute_objective
                  .prepare(
                    ref_dict,
                    predict_dict,
                    argument_dict,
                    LAMMPS))

----------------------------------------------------------------------------------
###                           Part 4: initialize and start the optimization                                 
----------------------------------------------------------------------------------
#### Purposes: 
* Perform optimization based on the input of the objective function and input file arguments.
* The following optimizer class can be used independently if the input file and objective functions are provided. 

#### Return: 
* optimize_fm: A Python instance initialized by parsing the input file arguments and take the objective function
* optimization starts by calling the method "run_optimization(), which implements an in-house Nelder-Mead simplex algorithm"

In [ ]:


optimize_fm = (optimizer
               .gradient_free
               .NelderMeadSimplex(
                   INPUT,
                   eval_objective,
                   skipped=last_line,
                   Output=JOBID+"/Output"))

# run optimization ...
# Don't run for the tutorial
optimize_fm.run_optimization()
 
